In [2]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer


In [23]:
df= pd.read_csv(r"C:\Users\U6056186\Downloads\train (1).csv",usecols=['Age','Fare','Survived'])

In [24]:
df.columns

Index(['Survived', 'Age', 'Fare'], dtype='object')

In [25]:
df.isna().sum()

Survived      0
Age         177
Fare          0
dtype: int64

In [28]:
df.dropna(inplace=True)

In [29]:
df.shape

(714, 3)

In [32]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [34]:
X=df[['Age','Fare']]
y=df['Survived']

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [36]:
X_train.head()

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000


In [37]:
clf = DecisionTreeClassifier()

In [38]:
clf.fit(X_train,y_train)
y_pred= clf.predict(X_test)

In [42]:
cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy').mean()

0.6330790297339594

In [70]:
age_kbn= KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
fare_kbn= KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [71]:
trf = ColumnTransformer([
    ('f1',age_kbn,[0]),
    ('f2',fare_kbn,[1])
        ])

In [72]:
X_train_kbn = trf.fit_transform(X_train)
X_test_kbn= trf.fit_transform(X_test)

In [73]:
trf.named_transformers_

{'f1': KBinsDiscretizer(encode='ordinal', n_bins=15),
 'f2': KBinsDiscretizer(encode='ordinal', n_bins=15)}

In [74]:
trf.named_transformers_['f1'].n_bins_

array([15])

In [78]:
trf.named_transformers_['f1'].bin_edges_[0]

array([ 1.        ,  9.        , 15.9       , 17.        , 20.        ,
       21.        , 24.        , 26.        , 28.        , 30.1       ,
       33.66666667, 36.        , 38.6       , 44.06666667, 52.        ,
       62.        ])

In [76]:
trf.named_transformers_['f2'].bin_edges_

array([array([  0.        ,   7.17166667,   7.77333333,   7.925     ,
                8.05      ,   9.5       ,  10.5       ,  13.        ,
               17.69333333,  22.62      ,  26.35276667,  31.275     ,
               38.1       ,  53.22666667,  79.27666667, 512.3292    ])],
      dtype=object)

In [86]:
pd.DataFrame({'age':X_train['Age'],'KbnAge':X_train_kbn[:,0],
              'AgeRange':trf.named_transformers_['f1'].bin_edges_[0].tolist(),
              'Fare':X_train['Fare'],'KbnFare':X_train_kbn[:,-1],
              'FareRanfe':trf.named_transformers_['f2'].bin_edges_[0].tolist()               
             })

ValueError: All arrays must be of the same length

In [96]:
output= pd.DataFrame({'age':X_train['Age'],'KbnAge':X_train_kbn[:,0],
             'Fare':X_train['Fare'],'KbnFare':X_train_kbn[:,-1],              
             })

In [97]:
output.head()

,age,KbnAge,Fare,KbnFare
328,31.0,8.0,20.5250,8.0
73,26.0,6.0,14.4542,7.0
253,30.0,8.0,16.1000,7.0
719,33.0,9.0,7.7750,2.0
666,25.0,6.0,13.0000,6.0


In [98]:
output['Age_binned']= pd.cut(X_train['Age'],
       bins=trf.named_transformers_['f1'].bin_edges_[0].tolist())

In [99]:
output.head()

,age,KbnAge,Fare,KbnFare,Age_binned
328,31.0,8.0,20.5250,8.0,"(30.1, 33.667]"
73,26.0,6.0,14.4542,7.0,"(24.0, 26.0]"
253,30.0,8.0,16.1000,7.0,"(28.0, 30.1]"
719,33.0,9.0,7.7750,2.0,"(30.1, 33.667]"
666,25.0,6.0,13.0000,6.0,"(24.0, 26.0]"


In [100]:
output['Fare_binned']= pd.cut(X_train['Fare'],
       bins=trf.named_transformers_['f2'].bin_edges_[0].tolist())

In [123]:
output.head()

,age,KbnAge,Fare,KbnFare,Age_binned,Fare_binned
328,31.0,8.0,20.5250,8.0,"(30.1, 33.667]","(17.693, 22.62]"
73,26.0,6.0,14.4542,7.0,"(24.0, 26.0]","(13.0, 17.693]"
253,30.0,8.0,16.1000,7.0,"(28.0, 30.1]","(13.0, 17.693]"
719,33.0,9.0,7.7750,2.0,"(30.1, 33.667]","(7.773, 7.925]"
666,25.0,6.0,13.0000,6.0,"(24.0, 26.0]","(10.5, 13.0]"
